## Regregression logistique

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pickle

# Charger les données
file = open('data/raw/dataset.pkl', 'rb')
DATA = pickle.load(file)
pheno = DATA['pheno']
X_gpa = DATA['X_gpa']
X_snps = DATA['X_snps']
X_genexp = DATA['X_genexp']

In [5]:
X_genexp

array([[-1.12876069, -0.81779089, -1.69142345, ...,  1.02347764,
        -1.15085433,  0.24994568],
       [-1.12876069, -0.35287294, -2.53190989, ..., -0.99980795,
        -1.18807964, -1.2716658 ],
       [ 0.26680539, -0.39662847, -0.95591998, ..., -0.22666416,
         0.18241751, -0.3702671 ],
       ...,
       [-0.43097765, -0.58287845, -0.99612165, ...,  0.35819654,
         0.37939308, -0.3702671 ],
       [ 1.45334338,  0.56409601,  1.02908939, ...,  0.2692959 ,
         0.06217161,  0.87786711],
       [-0.43097765,  0.34107776,  1.38943538, ..., -0.22666416,
         0.56298516,  0.56676242]])

In [2]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import recall_score

def refine_and_evaluate_model(pheno, X_gpa, X_snps, X_genexp, model, param_grid, model_name):
    performances = []
    hyperparams_records = []

    for antibiotique in pheno.columns[1:]:
        print(f"Traitement de l'antibiotique : {antibiotique}")

        # Extraire la cible (y) et les indices valides
        y = pheno[antibiotique].to_numpy()
        valid_indices = ~np.isnan(y)
        y = y[valid_indices]

        # Filtrer les matrices explicatives
        X_gpa_filtered = X_gpa[valid_indices]
        X_snps_filtered = X_snps[valid_indices]
        X_genexp_filtered = X_genexp[valid_indices]

        # Diviser les données en train/test avec stratification

        X_gpa_train, X_gpa_test, X_snps_train, X_snps_test, X_genexp_train, X_genexp_test, y_train, y_test = train_test_split(
            X_gpa_filtered, X_snps_filtered, X_genexp_filtered, y, 
            test_size=0.2, random_state=42, stratify=y
        )

        scaler_genexp = StandardScaler()
        X_genexp_train = scaler_genexp.fit_transform(X_genexp_train)  # Fit + transform sur l'entraînement
        X_genexp_test = scaler_genexp.transform(X_genexp_test)        # Transform uniquement sur le test

        # Construire les matrices combinées pour train et test
        X_train = np.hstack([X_gpa_train, X_snps_train, X_genexp_train])
        X_test = np.hstack([X_gpa_test, X_snps_test, X_genexp_test])

        # Grid Search pour Random Forest
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='recall', verbose=1)
        grid_search.fit(X_train, y_train)

        # Meilleur modèle
        meilleur_modele = grid_search.best_estimator_

        # Prédire sur l'ensemble de test
        y_pred = meilleur_modele.predict(X_test)

        # Calculer le macro recall
        rappel_macro = recall_score(y_test, y_pred, average='macro')
        print(f"Rappel macro pour {antibiotique} avec {model_name}: {rappel_macro:.4f}")

        # Identifier les coefficients non nuls
        coefficients = meilleur_modele.coef_[0]  
        non_zero_indices = np.where(coefficients != 0)[0]  

        colonnes_gpa = non_zero_indices[non_zero_indices < X_gpa_filtered.shape[1]]
        colonnes_snps = non_zero_indices[
            (non_zero_indices >= X_gpa_filtered.shape[1]) &
            (non_zero_indices < X_gpa_filtered.shape[1] + X_snps_filtered.shape[1])
        ] - X_gpa_filtered.shape[1]
        colonnes_genexp = non_zero_indices[
            non_zero_indices >= X_gpa_filtered.shape[1] + X_snps_filtered.shape[1]
        ] - (X_gpa_filtered.shape[1] + X_snps_filtered.shape[1])

        # Stocker les performances pour chaque antibiotique
        performances.append({
            'Antibiotique': antibiotique,
            'Recall': rappel_macro,
            'Colonnes_GPA': colonnes_gpa.tolist(),
            'Colonnes_SNPs': colonnes_snps.tolist(),
            'Colonnes_Expression_Genetique': colonnes_genexp.tolist()
        })

        # Stocker les hyperparamètres
        hyperparams_records.append({
            'Antibiotique': antibiotique,
            'Meilleur_paramètre': grid_search.best_params_,
            'Valeurs_des_hyperparamètres': param_grid
        })

    performances_df = pd.DataFrame(performances)

    # fichier CSV 
    
    csv_filename = f'result_{model_name}.csv'
    performances_df.to_csv(csv_filename, index=False, sep=';')

    hyperparams_df = pd.DataFrame(hyperparams_records)

    # Générer un fichier CSV pour les hyperparamètres
    hyperparams_csv_filename = f'param_{model_name}.csv'
    hyperparams_df.to_csv(hyperparams_csv_filename, index=False, sep=';')

    return performances_df

Au vu de la grande dimension p >> n et de la multicolinéarité, on choisit en premier lieu de pénaliser.
L'étude préalable a montré que seulement quelques variables semblent être utiles.

In [3]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', penalty='l1', max_iter=500)
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Paramètres pour la régularisation
    'class_weight': ['balanced', None]
}

performances_df = refine_and_evaluate_model(pheno, X_gpa, X_snps, X_genexp, model, param_grid, "LogisticRegression_L1")

print("\nPerformances du modèle:")
print(performances_df)

Traitement de l'antibiotique : Tobramycin
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Rappel macro pour Tobramycin avec LogisticRegression_L1: 0.9540
Traitement de l'antibiotique : Ceftazidim
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Rappel macro pour Ceftazidim avec LogisticRegression_L1: 0.8209
Traitement de l'antibiotique : Ciprofloxacin
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Rappel macro pour Ciprofloxacin avec LogisticRegression_L1: 0.8562
Traitement de l'antibiotique : Meropenem
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Rappel macro pour Meropenem avec LogisticRegression_L1: 0.8683
Traitement de l'antibiotique : Colistin
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Rappel macro pour Colistin avec LogisticRegression_L1: 0.7687

Performances du modèle:
    Antibiotique    Recall                                       Colonnes_GPA  \
0     Tobramycin  0.953984                [14347, 15032, 15